In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install openai

from openai import OpenAI
import pandas as pd
import numpy as np

# Set up OpenAI API key
client = OpenAI(
    api_key="$OPENAI_API_KEY",
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 13.2 MB/s eta 0:00:00


In [2]:
data = pd.read_csv('/kaggle/input/gdelt-climate-2023/bquxjob_25ad9db8_19346caa27a.csv')
data.shape

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/gdelt-climate-2023/bquxjob_25ad9db8_19346caa27a.csv'

In [ ]:
# For using a subset of the data
sample_size = max(1, int(0.22 * len(data)))
data = data.sample(n=sample_size, random_state=42)
data.shape

In [ ]:
data

In [4]:
def detect_conflict(headline1, headline2):
    prompt = f"""Given the following two headlines, classify their relationship as one of the following (the purpose is to build a dataset for a news source credibility network via conflict detection):
    - "conflicting" if the headlines provide contradictory information.
    - "consistent" if the headlines provide complementary or similar information.
    - "unrelated" if the headlines discuss different topics.

    Headline 1: "{headline1}"
    Headline 2: "{headline2}"

    Relationship:"""
    
    # Call the OpenAI API
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o-mini",
    )
    
    result = response.choices[0].message.content.strip().lower()
    
    # Normalize output for consistency
    if "conflict" in result:
        return "conflicting"
    elif "consistent" in result:
        return "consistent"
    elif "unrelated" in result:
        return "unrelated"
    else:
        return "unknown"  # Fallback for unexpected results

In [5]:
stuff = []
for i in range(1000):
    stuff.append(detect_conflict(i, i))
stuff

KeyboardInterrupt: 

# Create pairs of headlines to analyze, skipping comparisons from the same source

In [ ]:
pairs = []
for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if data.iloc[i]['domain'] != data.iloc[j]['domain']:  # Skip same source
            pairs.append((data.iloc[i]['title'], data.iloc[j]['title'], data.iloc[i]['desc'], data.iloc[j]['desc'], data.iloc[i]['domain'], data.iloc[j]['domain']))

In [ ]:
len(pairs)

In [ ]:
conflicts = []
for pair in pairs:
    headline1, headline2, desc1, desc2, source1, source2 = pair
    conflict_score = detect_conflict(f"Headline: {headline1} | Description: {desc1}", f"Headline: {headline2} | Description: {desc2}")
    conflicts.append({
        'source1': source1,
        'source2': source2,
        'headline1': headline1,
        'headline2': headline2,
        'conflict_score': conflict_score
    })

# Convert conflicts to a DataFrame for analysis
conflict_df = pd.DataFrame(conflicts)

# DO API CALLS IN PARALLEL FOR FASTER

In [ ]:
pd.set_option('display.max_colwidth', None)
filtered_conflict_df = conflict_df[conflict_df['conflict_score'] != 'unrelated']
filtered_conflict_df

In [ ]:
# Save DataFrame as a CSV file
filtered_conflict_df.to_csv('conflict_scores.csv', index=False)